In [ ]:
!pip install importlib-metadata==3.2.0
!python -m spacy download en_core_web_sm
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz!pip install PyPDF2
!pip install PyPDF2
!pip install pdfminer.six
!pip install PyMuPDF
!pip install tika

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.1.0
    Uninstalling importlib-metadata-6.1.0:
      Successfully uninstalled importlib-metadata-6.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.4.3 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.2.0 which is incompatible.
gym 0.25.2 requires importlib-metadata>=4.8.0; python_version < "3.10", but you have importlib-metadata 3.2.0 which is incompatible.
flask 2.2.3 requires importlib-metadata>=3.6.0; python_version < "3.10", but you have importlib-metadata 3.2.0 which is incompatible.
2023-04-06 13:18:49.353422: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use 

In [ ]:
import PyPDF2
import nltk
import os
import re
import string
import spacy
import json
from tika import parser
import fitz
from operator import itemgetter
from bs4 import BeautifulSoup

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('maxent_ne_chunker')
nltk.download('words')


from pdfminer.high_level import extract_text
from spacy.matcher import Matcher

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
!rm -r '/content/extracted_data'
!rm -r '/content/text_resumes'
!rm -r '/content/json_data'
!rm -r '/content/headings'

In [ ]:
!mkdir '/content/text_resumes'
!mkdir '/content/extracted_data'
!mkdir '/content/json_data'
!mkdir '/content/headings'

In [ ]:
class ResumeInformationExtractor:
  def __init__(self, directory_path):
    self.directory = directory_path
    self.text_directory = '/content/text_resumes'
    self.block_directory = '/content/extracted_data'
    self.json_directory = '/content/json_data'
    self.headings_directory = '/content/headings'
    self.store_text()
    print("DONE")

  def fonts(self, doc, granularity=False):
    styles = {}
    font_counts = {}
    for page in doc:
      blocks = page.get_text("dict")["blocks"]
      for b in blocks:
          if b['type'] == 0:
              for l in b["lines"]:
                  for s in l["spans"]:
                      if granularity:
                          identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                          styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                'color': s['color']}
                      else:
                          identifier = "{0}".format(s['size'])
                          styles[identifier] = {'size': s['size'], 'font': s['font']}

                      font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles

  def font_tags(self, font_counts, styles):
    p_style = styles[font_counts[0][0]]
    p_size = p_style['size']
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag


  def headers_para(self, doc, size_tag):
    header_para = []
    first = True
    previous_s = {}

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if b['type'] == 0:
                block_string = ""
                for l in b["lines"]:
                    for s in l["spans"]:
                        if s['text'].strip():
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:
                                    if block_string and all((c == "|") for c in block_string):
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        block_string = size_tag[s['size']] + s['text']
                                    else:
                                        block_string += " " + s['text']
                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']
                                previous_s = s
                    block_string += "|"
                header_para.append(block_string)
    return header_para



  def get_headings(self, elements):
    # resume_headings = ['Personal Information', 'Education', 'Academic Projects', 'Academic Achievements and Awards', 'Technical Skills', 'Work Experience', 'Projects', 'Certifications', 'Publications', 'Skills', 'Achievements and Awards', 'Volunteering', 'Extracurricular Activities', 'Leadership', 'Hobbies', 'References', 'Achievements', 'Experience']
    headings_ = []
    for item in elements:
        if item.startswith(('<h1>', '<h2>')):
            headings_.append(item)
    headings = []
    for item in headings_:
        if '>' in item and len(item.split('>')) > 1:
            text = item.split('>')[1].split('|')[0].strip()
            headings.append(text)
    filtered_words = []
    for word in headings:
      word = ''.join(e for e in word if e.isalnum() or e in ['/', '\\', ' '])
      filtered_words.append(word)
    # filtered_words = [word for word in headings if word.upper() in resume_headings]
    return filtered_words



  def extract_data(self, text , headings):
    regex_patterns = {}
    for heading in headings:
        regex_patterns[heading] = re.compile(r'^' + heading, re.IGNORECASE)

    text_blocks = {}
    for heading in headings:
        text_blocks[heading] = ''

    current_heading = None
    i = 0

    for line in text.split('\n'):
        found_heading = False
        for heading, regex_pattern in regex_patterns.items():
            if regex_pattern.match(line):
                current_heading = heading
                found_heading = True
                break
        if found_heading:
            continue

        if current_heading is not None:
            text_blocks[current_heading] += line + '\n'
    return text_blocks

  def get_file_content(self, doc):
    # parsed = parser.from_file(path)
    # result = parsed['content']
    # if not result:
    #   return result
    # else:
    #   result = "\n".join([line for line in result.split("\n") if line.strip() != ""])
    header_para = []
    first = True
    previous_s = {}

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if b['type'] == 0:
                block_string = ""
                for l in b["lines"]:
                    for s in l["spans"]:
                        if s['text'].strip():
                            if first:
                                previous_s = s
                                first = False
                                block_string = s['text']
                            else:
                                if s['size'] == previous_s['size']:
                                    if block_string and all((c == "|") for c in block_string):
                                        block_string = s['text']
                                    if block_string == "":
                                        block_string = s['text']
                                    else:
                                        block_string += " " + s['text']
                                else:
                                    header_para.append(block_string)
                                    block_string = s['text']
                                previous_s = s
                header_para.append(block_string)
    result = '\n'.join(header_para)
    return result

  def store_text(self):
    print(self.directory)
    files = sorted(os.listdir(self.directory))

    for file in files:
        path = self.directory + '/' + file
        doc = fitz.open(path)
        text_data = self.get_file_content(doc)
        if not text_data:
          continue
        with open(self.text_directory + '/' + file.split('.')[0] + '.txt', 'w') as f:
            f.write(text_data)

        font_counts, styles = self.fonts(doc, granularity=False)
        size_tag = self.font_tags(font_counts, styles)
        elements = self.headers_para(doc, size_tag)
        headings = self.get_headings(elements)
        with open(self.headings_directory + '/' + file.split('.')[0] + '.txt', 'w') as f:
          f.write(str(headings))

        text_blocks = self.extract_data(text_data , headings)
        with open(self.block_directory + '/' + file.split('.')[0] + '.txt', 'w') as f:
            for heading, text_block in text_blocks.items():
                if text_block:
                    f.write(heading.upper() + '\n')
                    f.write(text_block + '-' * 50 + '\n')

        with open(self.json_directory + '/' + file.split('.')[0] + '.json', 'w') as f:
            non_empty_text_blocks = {key: value.replace('\n', ' ') for key, value in text_blocks.items() if value}
            json.dump(non_empty_text_blocks, f, indent=2)
    return


In [ ]:
RESUME_DIRECTORY = '/content/type2'
ResumeInformationExtractor(RESUME_DIRECTORY)
!zip -r /content/json_data2.zip /content/json_data

/content/type2
DONE
updating: content/json_data/ (stored 0%)
updating: content/json_data/pdf_resume_121.json (deflated 51%)
updating: content/json_data/pdf_resume_737.json (deflated 43%)
updating: content/json_data/pdf_resume_131.json (stored 0%)
updating: content/json_data/pdf_resume_437.json (stored 0%)
updating: content/json_data/pdf_resume_387.json (deflated 49%)
updating: content/json_data/pdf_resume_166.json (stored 0%)
updating: content/json_data/pdf_resume_734.json (deflated 47%)
updating: content/json_data/pdf_resume_33.json (deflated 46%)
updating: content/json_data/pdf_resume_453.json (stored 0%)
updating: content/json_data/pdf_resume_564.json (deflated 48%)
updating: content/json_data/pdf_resume_957.json (stored 0%)
updating: content/json_data/pdf_resume_643.json (deflated 47%)
updating: content/json_data/pdf_resume_895.json (deflated 47%)
updating: content/json_data/pdf_resume_626.json (deflated 51%)
updating: content/json_data/pdf_resume_217.json (deflated 46%)
updating: 

In [ ]:
import os
os.removedirs('/content/type2/.ipynb_checkpoints')